In [9]:
# !wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/minsearch.py
# !wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

In [24]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import minsearch
import json
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [4]:
with open("documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course in docs_raw:
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"]
    ,keyword_fields=["course"]
)

index.fit(documents)

In [6]:
q = "the course has alraedy started, can I still enroll?"

boost = {"question": 3.0, "section": 0.5}

results = index.search(q
                       , boost_dict=boost
                       , num_results=5
                       , filter_dict={"course": "data-engineering-zoomcamp"})

results                       

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [7]:
load_dotenv(".env")

True

In [8]:
client = OpenAI()

In [9]:
def search(query: str) -> list:
    boost = {"question": 3.0, "section": 0.5}
    results = index.search(
        query
        , boost_dict=boost
        , num_results=5
        , filter_dict={"course": "data-engineering-zoomcamp"}
    ) 
    
    return results

In [42]:
def elastic_search(query: str) -> list:
    search_query = {
    "size": 5
    ,"query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query
                    ,"fields": ["question^3", "text", "section"]
                    ,"type": "best_fields"
                }
            }
            ,"filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}
    results = []
    response = es_client.search(index=index_name, body=search_query)
    for hit in response['hits']['hits']:
        results.append(hit['_source'])

    return results

In [19]:
es_client = Elasticsearch("http://localhost:9200")

In [10]:
def build_prompt(question: str, search_results: list) -> str:
    
    question = question.strip()

    # Prompt template format
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT, using only facts from the CONTEXT. 

    QUESTION: {question}
    CONTEXT: 
    {context}
    """.strip()

    # build the context string 
    context = ""
    
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=question, context=context)
    return prompt



In [11]:
def llm(prompt: str) -> str:
    response=client.chat.completions.create(
        model='gpt-4o'
        ,messages=[{'role': 'user', 'content': prompt}]
        )
    # print(prompt)
    return response.choices[0].message.content

In [34]:
query = "How do I run Kafka?"
def rag(query: str) -> str:
    search_results = search_es(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer   


In [40]:
rag(query)

'To run Kafka, in the project directory, execute the following command:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nThis command is for running a Java Kafka producer, but similar structures can be followed for other components like consumers or kstreams as per your specific implementation.'

In [23]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {
                "type": "text"
            },
            "section": {
                "type": "text"
            },
            "question": {
                "type": "text"
            },
            "course": {
                "type": "keyword"
            }
       }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})